In [1]:
"""
This Jupyter notebook implements the code from the
paper "Unveiling the Complex Morphologies of Sessile Droplets on Heterogeneous Surfaces"
for a square-pattern substrate.
The notebook defines several global constants that describe the shape of the substrate.
These constants typically do not need to be modified.
Following the flow chart in Fig. S2, a good entry point for the code is the function Main_M.
This function calculates the radius matrix for a given droplet volume v
and parameters m and n using Radius2fun.
The radius matrix is then processed by average_mainfun,
which computes the droplet height H and the droplet surface area under specific m and n constraints.
By looping over a range of m and n values for a given droplet volume,
the surface area matrix is obtained. The minimum value in the surface area matrix, S_min, is identified,
along with the corresponding parameters m and n.

Key Functions:
Main_M: Entry point for calculating the radius matrix.
Radius2fun: Computes the radius matrix for given parameters.
average_mainfun: Calculates the droplet height and surface area.

Results:
The minimum surface area S_min and the corresponding parameters m and n
are identified, providing insights into the morphology of sessile droplets on square substrat
"""


'\nThis Jupyter notebook implements the code from the\npaper "Unveiling the Complex Morphologies of Sessile Droplets on Heterogeneous Surfaces"\nfor a square-pattern substrate.\nThe notebook defines several global constants that describe the shape of the substrate.\nThese constants typically do not need to be modified.\nFollowing the flow chart in Fig. S2, a good entry point for the code is the function Main_M.\nThis function calculates the radius matrix for a given droplet volume v\nand parameters m and n using Radius2fun.\nThe radius matrix is then processed by average_mainfun,\nwhich computes the droplet height H and the droplet surface area under specific m and n constraints.\nBy looping over a range of m and n values for a given droplet volume,\nthe surface area matrix is obtained. The minimum value in the surface area matrix, S_min, is identified,\nalong with the corresponding parameters m and n.\n\nKey Functions:\nMain_M: Entry point for calculating the radius matrix.\nRadius2fu

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
from scipy.optimize import fsolve 
import sys
import time
from scipy import optimize
from numpy import savetxt

In [3]:
def intercept(s, N):
    """
    Calculates the initial radius of the inner tangent circle of an N-sided polygon.

    Args:
        s (float): The constant area of the substrate.
        N (int): The number of sides of the polygon (N >= 3).

    Returns:
        r (float): The radius of the inner tangent circle of the N-sided polygon, denoted as R in the paper.
    """
    r = np.sqrt(s/(np.tan(np.pi/N)*N))
    return r

In [4]:
N = 4 # Initial number of sides for the square pattern
area = 1 # Liuqid-solid contact area/substrate area, denoted as S_1 in paper
theta_ = (2*np.pi)/N # The interior angle, denoted as \varphi in paper
r = intercept(area, N) # The radius of the inner tangent circle for the N-polygon pattern 
c_max = np.power(-1, N)*np.power(-r,N) # Mathematical transformations for easy implementation
###### coefficients in Eq.S6:
a1 = np.sin(theta_)
b1 = np.cos(theta_)
a2 = np.sin(2*theta_)
b2 = np.cos(2*theta_)
a3 = np.sin(3*theta_)
b3 = np.cos(3*theta_)
a4 = np.sin(4*theta_)
b4 = np.cos(4*theta_)
#####

In [5]:
def r2fun(R, *data):#polar coordinate
    """
    This function describes the shape of a droplet on an N-polygon substrate, 
    as specified by Equation S5. It will be submitted to "r_solution_initial" or "r_solution_array"
    to solve for the radius matrix given a droplet volume "v", and parameter "m".
    Args:
        R (float): The radius in the polar coordinate system.
        theta, c, m (tuple):
            theta (float): The angle in the polar coordinate system.
            c (float): A parameter for the scaling factor "f(c)" as defined in the paper.
            m (float): Another parameter for the scaling factor "f(c)" as defined in the paper.
    Returns:
        The left side of Equation S5, to be used for solving the radius matrix.
    """

    global theta_, r, N, c_max
    theta, c, m= data
    f=1
    x = R*np.cos(theta)
    y = R*np.sin(theta)
    factor = (1 + (c/c_max))**m
    for echo in range(N):
        i = echo+1
        fi = ((np.sin(i*theta_))*(y/factor) + np.cos(i*theta_)*(x/factor))-r
        f = f*fi
    f = np.power(-1, N)*f
    return f-c

In [6]:
def r_solution_initial(r0, theta, c, m):
    """
    This function uses "fsolve" to determine the precise radius "r"
    given an initial guess "r0", angle "theta", and parameters "c" and "m".

    Args:
        r0 (float): Initial guess for the radius.
        theta (float): The angle in the polar coordinate system.
        c (float): A parameter related to the scaling factor "f(c)" as described in the paper.
        m (float): Another parameter related to the scaling factor "f(c)" as described in the paper.
        
    Returns:
       (float): The exact radius corresponding to the given parameters, intended for use in the radius
       matrix radius[0][j].
    """

    data = (theta, c, m)
    r,_,ier,mesg = fsolve(r2fun, r0, args=data, full_output=True)
    r_value = r.item(0)
    if ier != 1:
        print ("r_solution:I coudn't solve the algebraic constraint, error:\n\n:",r_value, theta,c, m)
    sys.stdout.flush()
    #r_value = sol.x
    return r_value

In [7]:
def r_solution_array(r0, theta, c, m):
    """
    This function uses 'fsolve' to determine the precise radius 'r' given an initial guess 'r0', 
    an angle 'theta', and parameters 'c' and 'm'.

    Args:
        r0 (array): The initial guess for the radius, typically the previous value in the radius array.
        theta (float): The angle in the polar coordinate system.
        c (float): A scaling factor parameter, as described in the referenced paper.
        m (float): Another scaling factor parameter, as described in the referenced paper.
    Returns:
        array: The calculated radius array corresponding to the given parameters.
    """

    data = (theta, c, m)
    r,_,ier,mesg = fsolve(r2fun, r0, args=data, full_output=True)
    if ier != 1:
        print ("r_solution:I coudn't solve the algebraic constraint, error:\n\n:",c, m)
    sys.stdout.flush()
    return r

In [8]:
def Radius2fun(dimension,theta_r, c_r, m):
    """
    This function generates a radius matrix.

    Args:
        dimension (int): The dimensions of the radius matrix.
        theta_r (array-like): An array of angle ranges from 0 to 2π/N with a length of "dimension".
        c_r (array-like): An array of the parameter "c" ranging from 0 to c_max with a length of "dimension".
        m (float): A scaling factor parameter "f(c)" as described in the referenced paper.

    Returns:
        numpy.ndarray: A radius matrix of size "dimension" x "dimension" for the specified parameter "m".
"""

    global N, r
    r0 = r # the radius of the inner tangent circle for the N-polygon pattern
    radius = np.zeros(shape = (dimension, dimension))
    for i in range(dimension):
        c = c_r[i]
        if i <=1:
            for j in range(dimension):
                theta = theta_r[j]
                R = r_solution_initial(r0, theta, c, m)
                radius[i][j] = R
                r0 = R
        else:
            r0 = radius[i-1]
            R = r_solution_array(r0, theta_r, c, m)
            radius[i] = R
    return radius

In [9]:
def dev_r_theta2fun(theta_ranges, Radius, c_ranges, m):
    """
    Calculates the partial derivatives of the radius with respect to the angle
    (\(\partial R / \partial \theta\)), as described in Equation S18.

    Args:
        theta_ranges (array-like): Array of angle ranges from 0 to \(2\pi/N\), 
        with a length specified by the "dimension".
    Radius (array-like): Matrix representing the radius for a given parameter "m".
    c_ranges (array-like): Array of parameter "c" ranges from 0 to \(c_{\text{max}}\),
    with a length specified by the "dimension".
    m (float): Scaling factor parameter "f(c)" as described in the paper.
    
    Returns:
        numpy.ndarray: Matrix of partial derivatives of the radius with respect to the angle.
    """

    global r, c_max
    C_N_factor = np.reshape(((1 + c_ranges/c_max)**m), (-1,1))
    Sin = np.sin(theta_ranges)
    Cos = np.cos(theta_ranges)
    R_sin = Radius*Sin
    R_cos = Radius*Cos
    eq1 = a1*R_sin+b1*R_cos-r*C_N_factor
    eq2 = a2*R_sin+b2*R_cos-r*C_N_factor
    eq3 = a3*R_sin+b3*R_cos-r*C_N_factor
    eq4 = a4*R_sin+b4*R_cos-r*C_N_factor
############################
    d_eq1_1 = a1*Sin+b1*Cos
    d_eq2_1 = a2*Sin+b2*Cos
    d_eq3_1 = a3*Sin+b3*Cos
    d_eq4_1 = a4*Sin+b4*Cos
###########################
    d_eq1_2 = a1*R_cos-b1*R_sin
    d_eq2_2 = a2*R_cos-b2*R_sin
    d_eq3_2 = a3*R_cos-b3*R_sin
    d_eq4_2 = a4*R_cos-b4*R_sin
###
    z_denominator = (d_eq1_1*eq2*eq3*eq4+d_eq2_1*eq1*eq3*eq4+d_eq3_1
                     *eq2*eq1*eq4+d_eq4_1*eq1*eq3*eq2 )
    z_numerator = -(d_eq1_2*eq2*eq3*eq4+d_eq2_2*eq1*eq3*eq4+d_eq3_2
                     *eq2*eq1*eq4+d_eq4_2*eq2*eq3*eq1)
    dev_r_theta = z_numerator/z_denominator
    return dev_r_theta

In [10]:
def dev_r_C2fun(theta_ranges, Radius, c_ranges, m):
    """
    This function computes the partial derivatives of the radius (R)
    with respect to the parameter (c), as described in Equation S19.

    Args:
        theta_ranges (array): An array representing the angle ranges 
        from 0 to 2π/N with a specified length ("dimension").
        Radius (matrix): A matrix of radius values corresponding to a particular parameter "m".
        c_ranges (array): An array of parameter "c" values ranging from 0 to
        c_max with a specified length ("dimension").
        m (float): A scaling factor parameter "f(c)" as defined in the paper.

    Returns:
        matrix: A matrix containing the partial derivatives of the radius with respect to the parameter "c".
    """

    global r, c_max, N
    c = np.reshape(c_ranges, (-1,1))
    C_N_factor = (1+(c/c_max))**m
    Sin = np.sin(theta_ranges)
    Cos = np.cos(theta_ranges)
    R_sin = Radius*Sin
    R_cos = Radius*Cos
    eq1 = a1*R_sin+b1*R_cos-r*C_N_factor
    eq2 = a2*R_sin+b2*R_cos-r*C_N_factor
    eq3 = a3*R_sin+b3*R_cos-r*C_N_factor
    eq4 = a4*R_sin+b4*R_cos-r*C_N_factor
    ############################
    d_eq1_1 = a1*Sin+b1*Cos
    d_eq2_1 = a2*Sin+b2*Cos
    d_eq3_1 = a3*Sin+b3*Cos
    d_eq4_1 = a4*Sin+b4*Cos
    ###########################
    z_denominator = (d_eq1_1*eq2*eq3*eq4+d_eq2_1*eq1*eq3*eq4+d_eq3_1
                     *eq2*eq1*eq4+d_eq4_1*eq1*eq3*eq2)
    c_n_factor = m*((1+c/c_max)**(m-1))*(1/c_max)
    C_derv = C_N_factor**N+c*N*(C_N_factor**(N-1))*c_n_factor
    z_numerator = C_derv+r*c_n_factor*(eq2*eq3*eq4+eq1*eq3*eq4+eq1*eq2*eq4+eq1*eq2*eq3)
    dev_r_c = z_numerator/z_denominator
    return dev_r_c

In [11]:
def dev_h_c2fun(c, n):
    """
    This function computes the partial derivative of height (h) with respect to c,
    denoted as (\partial h)/(\partial c), which corresponds to the third part in Eq.S9 but without info of "H".

    Args:
        c (array): Array of parameter values ranging from 0 to c_max
        with 'dimension' number of elements.
        n (float): Parameter used in the droplet height definition h = H * c^n in the paper,
        where H is the droplet height.

    Returns:
    (array):Array of partial derivatives of height with respect to parameter c,
    scaled by a constant factor of H.
    """

    return (n/np.power(c_max, n))*np.power(c,n-1)

In [12]:
def average_mainfun(v, n, m, Radius, theta_ranges, C_ranges):
    """
    Calculates the height and the surface area of a droplet given its volume 'v' and parameters 'm' and 'n'.

    Args:
        v (float): Droplet volume.
        m (float): Scaling factor parameter 'f(c)' as defined in the paper.
        n (float): Parameter defining the droplet height 'h = Hc^n' in the paper, where 'H' is the droplet height.
        Radius (ndarray): Matrix of radii corresponding to parameter 'm'.
        theta_ranges (ndarray): Array defining angle ranges from 0 to 2π/N with 'dimension' length.
        C_ranges (ndarray): Array defining parameter 'c' ranges from 0 to c_max with 'dimension' length.

    Returns:
        H (float): Droplet height under parameters 'n' and 'm'.
        surface_size (float): Droplet surface area under parameters 'n' and 'm'.
    """

     
    global c_max, r, N

    dimen_theta = len(theta_ranges)
    dimen_c = len(C_ranges)
    
    dev_h_c = dev_h_c2fun(C_ranges, n)
    ds_base = (1/2)*(Radius**2)*(2*(np.pi)/(dimen_theta*N))
    ds_2 = np.delete(ds_base, -1, axis=1)
    ds_3 = np.delete(ds_base, 0, axis=1)

    delta_base_2 = np.sum(ds_2, axis=1)
    delta_base_3 = np.sum(ds_3, axis=1)
    
    dela_base_average = 0.5*(delta_base_2+delta_base_3)
    
    
    dv = (dela_base_average*dev_h_c)*(c_max/dimen_c)
    dv2 = np.delete(dv, -1, axis=0)
    dv3 = np.delete(dv, 0, axis=0)
    dv_average = 0.5*(dv2+dv3)
    
    dv_sum = np.sum(dv_average)
    H = v/(dv_sum*N)##### Eq.S13 the droplet height is determinded under given "n" and "m"
    
    dev_r_theta = dev_r_theta2fun(theta_ranges, Radius, C_ranges, m)
    dl = np.sqrt(Radius**2 + dev_r_theta**2)*(2*(np.pi)/(dimen_theta*N))####Eq.S15

    d_h = H*dev_h_c
    dh = d_h.reshape(-1,1)
    d_r_c = dev_r_C2fun(theta_ranges, Radius, C_ranges, m)
    
    #########################################################
    d_r_c[0][dimen_c//2] = 0 #when c=0, the d_r_c is not derivative, so to avoid the singularity
    #########################################################
    
    
    
    ds_surface = np.sqrt(dh**2+d_r_c**2)*(c_max/dimen_c)####Eq.S16
    surface = dl*ds_surface####the integral part in Eq.S14 

    
    surface1 = np.delete(surface, -1, axis=0)
    surface2 = np.delete(surface1, -1, axis=1)
    
    surface3 = np.delete(surface, 0, axis=0)
    surface4 = np.delete(surface3, 0, axis=1)
    
    surface = 0.5*(surface2+surface4)
    
    surface_size = np.sum(surface)*N
    return H, surface_size

In [13]:
def Main_M(v, dimen, m_ranges, n_ranges):
    """
    Calculates the minimum droplet surface area for a given volume.
    
    Args:
        v (float): Droplet volume.
        dimen (int): Dimension for radius matrix.
        m_ranges (array-like): Array of possible values for parameter "m".
        n_ranges (array-like): Array of possible values for parameter "n".
        
    Returns:
        surface_small (float): Minimum surface area of the droplet.
        height (float): Corresponding height when the minimum surface area is achieved.
        M_n_surface_matrix (ndarray): Surface area matrix under dimensions specified by m_ranges*n_ranges.
        m_ranges[index[0]] (float): Parameter "m" value corresponding to the minimum surface area.
        n_ranges[index[1]] (float): Parameter "n" value corresponding to the minimum surface area.
    """

    C_ranges = np.linspace(1e-16, 0.999*c_max, dimen)
    theta_ranges = np.linspace(0, (2*np.pi)/N, dimen)
    m_length = len(m_ranges)
    n_length = len(n_ranges)
    M_n_surface_matrix = np.zeros((m_length, n_length))
    M_n_height_matrix = np.zeros((m_length, n_length))
    for i in tqdm(range(m_length)):
        M_Radius = Radius2fun(dimen, theta_ranges, C_ranges, m_ranges[i])# raidus matrix for certain m
        for j in range(n_length):
            n_h, n_s = average_mainfun(v, n_ranges[j],m_ranges[i], M_Radius, theta_ranges, C_ranges)# droplet height and surface under certain m and n
            M_n_surface_matrix[i][j] = n_s
            M_n_height_matrix[i][j] = n_h
    surface_small = M_n_surface_matrix.min()
    index = np.where(M_n_surface_matrix == np.min(M_n_surface_matrix))
    height = M_n_height_matrix[index]
    return surface_small, height, M_n_surface_matrix, m_ranges[index[0]], n_ranges[index[1]]

#### Test for d_r_c

In [14]:
Volume = [0.06,0.5]#0.1, 0.14, 0.18,0.3, 0.5, 1, 1.5]

In [15]:
#m_ranges = np.arange(-3, 3, 0.1)
#n_ranges = np.arange(0.5,1, 0.01)
m_ranges = np.arange(-3, 3, 0.1)
n_ranges = np.arange(0.5,2, 0.01)

In [17]:
ss = []
hh = []
s_matrixs = []
m_indexs = []
n_indexs = []
Dimen = 200
for v in Volume:
    s, h, s_matrix, m,n= Main_M(v, Dimen, m_ranges, n_ranges)
    ss.append(s)
    hh.append(h)
    s_matrixs.append(s_matrix)
    m_indexs.append(m)
    n_indexs.append(n)

  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
m_indexs

[array([0.5]), array([1.])]

In [18]:
n_indexs

[array([1.56]), array([1.24])]

In [19]:
hh

[array([0.12556505]), array([0.66727434])]

In [20]:
ss

[1.0556323553690496, 2.483987963583553]

In [21]:
#surface_energy = s_matrixs[0]

In [22]:
#savetxt('test_squ_03.csv', surface_energy, delimiter=',')